By: Gabriel Peery

# The Question

TODO

The machinery to run the simulations was done outside of this notebook to make it more portable and be able to be invoked using the command line. Here are the two source files, an explanation of some implementation details follows:

In [ ]:
# simulate

In [ ]:
# time_logging

(Explain how they are invoked, some decisions made ie. Numba, folder creation)

# The Analysis

Many different simulations were run during the debugging process, but we'll focus on two here. The first was run over night using the following invocation:
(todo)
The second one was to demonstrate (todo):
(todo)

We'll read in the results here, write some code to make it easier to analyze, and then get to it.

In [2]:
from json import load
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [244]:
class SimResult:
    """Class to store results of a simulation, plot information about them. Best
    if used as return of read_results method.
    """
    
    def __init__(self,
        link_energy_predict : pd.DataFrame,
        link_time : pd.DataFrame,
        spin : pd.DataFrame,
        link : pd.DataFrame,
        meta : pd.DataFrame,
        config : dict,
        accept : pd.DataFrame
    ):
        """Stores information about a simulation, providing a convenient
        container for plotting.
        """
        self.link_energy_predict = link_energy_predict
        self.link_time = link_time
        self.spin = spin
        self.link = link
        self.meta = meta
        self.config = config
        self.accept = accept
        
    def plot_equilibration(self, temp_idx : int):
        """Uses PyPlot to display a plot of link overlap as predicted during
        equilibration and as observed in the simulation as a function of Monte
        Carlo sweep, for the data at some temperature index.
        """
        # Put stuff on the plot
        plt.xscale("log")
        sweep_numbers = -1 + (2**(np.arange(len(self.link_time.iloc[0])) + 9))
        link_from_energy = np.array(list(self.link_energy_predict.iloc[temp_idx]))
        link_observed = np.array(list(self.link_time.iloc[temp_idx]))
        plt.plot(sweep_numbers, link_from_energy, "o-r", label="Predicted")
        plt.plot(sweep_numbers, link_observed, "o-b", label="Observed")
        
        # Setup labels
        this_temp = self.meta["Temperature"][temp_idx]
        plt.title(
            f"Link Overlap vs. Monte Carlo Sweep Number at T={this_temp:.2}"
        )
        plt.xlabel("$N_{sweep}$")
        plt.ylim([0,1])
        plt.ylabel("$q_l$")
        plt.legend()
        plt.show()
        
    def plot_spin_dist(self, temp_idx : int):
        """Uses PyPlot to show a histogram of the spin overlaps observed during
        the simulation, for the states at some temperature determined by the
        index.
        """
        # Put on the plot
        q_values = np.linspace(-1, 1, len(self.spin.iloc[temp_idx]))
        freqs = self.spin.iloc[temp_idx]
        plt.bar(q_values, freqs, 1/len(self.spin.iloc[temp_idx]))
        
        # Setup labels
        this_temp = self.meta["Temperature"][temp_idx]
        plt.title(f"Distribution of Spin Overlap at T={this_temp:.2}")
        plt.xlim(-1.1, 1.1)
        plt.xlabel("$q$")
        plt.ylabel("$P(q)$")
        plt.show()
        
    def plot_link_dist(self, temp_idx : int):
        """Uses PyPlot to show a histogram of the link overlaps observed during
        the simulation, for the states at some temperature determined by the
        index.
        """
        # Put on the plot
        ql_values = np.linspace(-1, 1, len(self.link.iloc[temp_idx]))
        freqs = self.link.iloc[temp_idx]
        plt.bar(ql_values, freqs, 1/len(self.link.iloc[temp_idx]))
        
        # Setup labels
        this_temp = self.meta["Temperature"][temp_idx]
        plt.title(f"Distribution of Link Overlap at T={this_temp:.2}")
        plt.xlim(-1.1, 1.1)
        plt.xlabel("$q_l$")
        plt.ylabel("$P(q_l)$")
        plt.show()
        
    def plot_acceptance(self):
        """Uses PyPlot to display a plot of acceptance ratios versus temperature
        """
        # Put stuff on the plot
        temp = np.array(list(self.meta["Temperature"]))
        ratios = np.array(list(self.accept[0]))
        plt.plot(temp[:-1], ratios, "o-g")
        
        # Setup labels
        plt.title(
            "Acceptance Ratio vs. Temperature"
        )
        plt.xlabel("$T$")
        plt.ylabel("Acceptance Ratio")
        plt.show()
    
    
# NOTE: This only works for Windows, systems with '\' as directory separator
def read_results(name : str) -> SimResult:
    """Given a name of a simulation, reads in data from the directory.
    
    Returns corresponding DataFrame's and dictionary of its configuration in a
    special class.
    """
    link_energy_predict = pd.read_csv(
        name + r"\link_energy_predict.csv",
        header=None
    )
    link_time = pd.read_csv(
        name + r"\link_time.csv",
        header=None
    )
    spin = pd.read_csv(
        name + r"\spin.csv",
        header=None
    )
    link = pd.read_csv(
        name + r"\link.csv",
        header=None
    )
    meta = pd.read_csv(
        name + r"\meta.csv",
        index_col=0
    )
    with open(name + r"\config.json") as config_file:
        config = load(config_file)
    accept = pd.read_csv(
        name + r"\accept.csv",
        header=None
    )
    return SimResult(
        link_energy_predict,
        link_time,
        spin,
        link,
        meta,
        config,
        accept
    )

In [133]:
def full_suite_analysis(name : str):
    """Reads in a result, plots all the things"""
    sr = read_results(name)
    sr.plot_acceptance()
    for i in range(len(sr.meta.index)):
        sr.plot_equilibration(i)
    for i in range(len(sr.meta.index)):
        sr.plot_spin_dist(i)
    for i in range(len(sr.meta.index)):
        sr.plot_link_dist(i)
    return sr

Now, let's look at the big simulation:

In [251]:
# TODO

(todo, including the other one)

## Conclusion

todo

# Future Directions

todo